# 1. Set up project in GCP
Using console.cloud.google.com, I set up a project called grav-waves. This project was automatically assigned a suffix (358320) by Google (most probably because this project exists somewhere else already on GCP).

# 2. Generate owner JSON key
Next, I created a service account and generated a [JSON key with owner access to that service account](https://cloud.google.com/iam/docs/creating-managing-service-accounts).

# 3. Upload key
I uploaded data and the JSON key from step 2 to the MNtrafficER folder containing this and the other scripts.

Next, I ran the code below to stand-up the database.

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import datetime

In [ ]:
# Mount drive - needed for GBQ creds
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_path = '/content/drive/MyDrive/GitHub'

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
    f'{root_path}/MNtrafficER/key/mn-dps-incidents-03da8ed88c61.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

# 4. Import DPS extracts & clean

In [ ]:
path_prefix = '/content/drive/MyDrive/GitHub/MNtrafficER/data'

incidents = pd.read_csv(f'{path_prefix}/incidents_2023-02-27 22_13_03.csv')
vehicles = pd.read_csv(f'{path_prefix}/vehicles_2023-02-27 22_13_03.csv')

# Clean up date fields and drop extraneous columns
incidents['incident_date_time'] = pd.to_datetime(incidents['date_time'], infer_datetime_format = True)
incidents['last_updated'] = pd.to_datetime(incidents['last_updated'], infer_datetime_format = True)
incidents = incidents.drop(columns = ['date_time'])

In [ ]:
incidents.head()

,addtl_info,contact,description,district,icr,incident_type,last_updated,location,outside_agencies,road_condition,incident_date_time
0,This fatal crash involved an unbelted/unrestra...,MSP PIO Lt. Gordon Shank (651) 343-4680,A Nissan Maxima was at a high rate of speed No...,2400 East Metro,23401089,Fatal,2023-02-11 05:51:00,"Northbound Highway 61 North of Lower Afton Rd,...","Ramsey County SO, St Paul PD, Prescott PD (WI)",Dry,2023-02-10 23:08:00
1,Information believed complete,Sgt. Jesse Grabow (218) 639-3168,Semi was eastbound on I-94 near milepost 65 wh...,2900 Detroit Lakes,23900275,Injury,2023-02-13 11:18:00,"EB I-94 MP64, TUMULI TWP, Otter Tail County",Otter Tail County and Dalton Fire Department,Dry,2023-02-11 15:50:00
2,Information believed complete,MSP PIO Lt. Gordon Shank (651) 343-4680,A Chevrolet Impala was southbound on Highway 5...,3100 Virginia,23310194,Injury,2023-02-12 01:52:00,"Hwy 53 just north of Willow River Road , LEIDI...","Orr Fire, Cook Ambulance, St Louis County",Dry,2023-02-11 23:03:00
3,Information believed complete,MSP PIO Lt. Gordon Shank (651) 343-4680,The Ford Fusion was southbound in the northbou...,2400 East Metro,23401120,Fatal,2023-02-12 15:27:00,"Northbound US Highway 61 and Jamaica Avenue , ...","Cottage Grove PD, Cottage Grove Fire and EMS, ...",Dry,2023-02-12 08:13:00
4,Information believed complete,Sgt. Troy Christianson (507) 923-2050,The Chevrolet truck was northbound on US Hwy 7...,2300 Marshall,23300420,Injury,2023-02-12 16:13:00,"US Hwy 75 at 211th St in Pipestone county, ALT...","Pipestone CSO, Lake Benton FD, Pipestone Amb, ...",Dry,2023-02-12 13:29:00


In [ ]:
vehicles.head()

,gender,person_type,age,airbag_deployed,alcohol_involved,health_care_facility,helmet,injury_type,location,name,seat_belt,vehicle,incident_case_record
0,NaN,driver,NaN,Yes,NaN,Regions Hospital,Not Applicable,NaN,NaN,Male Age: 16,Yes,2008 Nissan Maxima,23401089
1,male,passenger,17,Yes,Unknown,Ramsey County Medical Examiner,Not Applicable,Fatal,"Prescott, WI, USA",Cadence Lee Allen,No,2008 Nissan Maxima,23401089
2,male,passenger,17,Yes,Yes,Regions Hospital,Not Applicable,Non-life Threatening,"Prescott, WI, USA",Bryndan Anthony Hartman,Yes,2008 Nissan Maxima,23401089
3,male,passenger,17,Yes,Unknown,Ramsey County Medical Examiner,Not Applicable,Fatal,"Prescott, WI, USA",Aiden Patrick Nelson,No,2008 Nissan Maxima,23401089
4,female,driver,66,No,No,Fergus Falls Hospital,Not Applicable,Non-life Threatening,NaN,Susan Recknell,Yes,2022 Freightliner Semi,23900275


# 5. Push dataframes to GCP
Create a dataset called dps within the project (mn-dps-incidents). Upload the newly cleaned dataframe to the mn-dps-incidents.dps.incidents table. Both the dataset and table are persistent, so this code only needs to be run once.

For help specifying the schema, see [these instructions](https://cloud.google.com/bigquery/docs/schemas).

In [ ]:
# Create DPS dataset
dps = bigquery.Dataset('mn-dps-incidents.dps')
dps.location = 'US'
dps = client.create_dataset(dps)

In [ ]:
job_config = bigquery.LoadJobConfig(
    autodetect = True,
    source_format = bigquery.SourceFormat.CSV)

In [ ]:
job = client.load_table_from_dataframe(incidents, 'mn-dps-incidents.dps.incidents', job_config = job_config)

In [ ]:
job = client.load_table_from_dataframe(vehicles, 'mn-dps-incidents.dps.vehicles', job_config = job_config)

# 6. Test load

In [ ]:
metro_q = '''SELECT *
FROM dps.incidents
WHERE district = '2400 East Metro';'''

metro_df = client.query(metro_q).to_dataframe()

In [ ]:
metro_df

,addtl_info,contact,description,district,icr,incident_type,last_updated,location,outside_agencies,road_condition,incident_date_time
0,This fatal crash involved an unbelted/unrestra...,MSP PIO Lt. Gordon Shank (651) 343-4680,A Nissan Maxima was at a high rate of speed No...,2400 East Metro,23401089,Fatal,2023-02-11 05:51:00,"Northbound Highway 61 North of Lower Afton Rd,...","Ramsey County SO, St Paul PD, Prescott PD (WI)",Dry,2023-02-10 23:08:00
1,Information believed complete,MSP PIO Lt. Gordon Shank (651) 343-4680,The Ford Fusion was southbound in the northbou...,2400 East Metro,23401120,Fatal,2023-02-12 15:27:00,"Northbound US Highway 61 and Jamaica Avenue , ...","Cottage Grove PD, Cottage Grove Fire and EMS, ...",Dry,2023-02-12 08:13:00
2,Information believed complete,MSP PIO Lt. Gordon Shank (651) 343-4680,The Porche Panamera was traveling at a high ra...,2400 East Metro,23401299,Injury,2023-02-20 02:23:00,North Bound Cleveland Ave to West Bound Hwy 36...,"Ramsey County Sheriff, Roseville Police Depart...",Dry,2023-02-19 23:51:00
3,Information believed complete,MSP PIO Lt. Gordon Shank (651) 343-4680,Nissan Rogue and Buick Lacrosse were traveling...,2400 East Metro,23401409,Fatal,2023-02-27 14:35:00,"Northbound Highway 77 over I-35E, EAGAN, Dakot...","Dakota County Sheriffs, Eagan Police, Eagan Fi...",Snow/Ice,2023-02-22 08:39:00
